<a href="https://colab.research.google.com/github/akashjorss/topic_modelling/blob/main/comparison_deep_TM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#different versio of numpy required for top2vec
!pip uninstall numpy
!pip install numpy==1.19.5

In [68]:
#install the packages
# !pip install sentence-transformers
# !pip install nltk
# !pip install torch
# !pip install tensorflow
# !pip install tensorboard
# !pip install top2vec
# !pip install top2vec[sentence_encoders]
# !pip install top2vec[sentence_transformers]
# !pip install torch sentence_transformers
# !pip install top2vec[indexing]
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
import torch


In [ ]:
#import required packages
import random
from tqdm import tqdm
import numpy as np
import os
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
!git clone https://github.com/akashjorss/sentence_bert
!unzip sentence_bert/sentences.zip
import pickle
with open('sentences.bin', 'rb') as f:
  sentences = pickle.load(f)

In [ ]:
# import tensorflow as tf
# import tensorflow_datasets as tfds
# from tensorboard.plugins import projector
# #load tensorboards with magics
# %tensorflow_version 2.x
# %load_ext tensorboard

In [ ]:
documents = random.choices(sentences,k = 100000)
documents = [d.lower() for d in documents]

In [69]:
sbert_model = SentenceTransformer('stsb-mpnet-base-v2')
torch.save(sbert_model, "./stsb-mpnet-base-v2.pt")

sample_data  sentence_bert  sentences.bin  stsb-mpnet-base-v2.pt


In [71]:
from top2vec import Top2Vec
# use = Top2Vec(documents, embedding_model='universal-sentence-encoder', workers=4, document_ids=list(range(0,len(documents))))
# sbert = Top2Vec(documents, embedding_model_path='./stsb-mpnet-base-v2.pt', workers=4, document_ids=list(range(0,len(documents)))) #download sentence sbert
# doc2vec = Top2Vec(documents, embedding_model='doc2vec', workers=4, document_ids=list(range(0,len(documents))), speed="deep-learn")

2021-06-02 14:46:20,598 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
2021-06-02 14:46:24,564 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2021-06-02 14:52:50,523 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2021-06-02 14:54:47,224 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2021-06-02 14:54:53,926 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


In [72]:
print("Total num of topics in USE:", use.get_num_topics())
print("Total num of topics in doc2vec:", doc2vec.get_num_topics())
print("Total num of topics in sentence-bert:", sbert.get_num_topics())

Total num of topics in USE: 430
Total num of topics in doc2vec: 1069
Total num of topics in sentence-bert: 1329


In [73]:
def calc_IG(words, docs):
  """calculate information gain for a topic in topic model containing words and documents
  args:
    words (list): list of top words in the topic. 
    docs (list): list of docs in the topic
  returns:
    ig (float): information gain of this topic with respect to total independence b/w words and docs"""
  #sanity check
  assert(len(words)!=0)
  assert(len(docs)!=0)
  words = list(set(words)) #drop the redundant words
  #convert words to lower case
  words = [word.lower() for word in words]
  #tokenize the docs using nltk punkt
  tokenized_docs = [word_tokenize(doc.lower()) for doc in docs]
  #create inverted index between words and docs
  inverted_index = np.zeros((len(words),len(docs)))
  for i in range(len(words)):
    for j in range(len(docs)):
      inverted_index[i][j] = tokenized_docs[j].count(words[i])
  # print(words)
  # print(inverted_index)
  ig = 0.0
  P_d = 1/len(docs)
  P_w = 1/len(words)
  for i in range(len(words)):
    for j in range(len(docs)):
      if np.sum(inverted_index[i]) != 0:
        P_dw = inverted_index[i][j]/np.sum(inverted_index[i]) #total number of documents in which w occurs
        if P_dw != 0:
          ig += P_dw*P_w*np.log2(P_dw/P_d)
  
  return ig     

#test
doc1 = "The big sharks of Belgium drink beer."
doc2 = "Belgium has great beer. They drink beer all the time."
doc3 = "They drink a lot of beer in Belgium"
doc4 = "cha cha"
words = ["so", "what"]
docs = [doc1, doc2, doc3, doc4]
calc_IG(words, docs)

0.0

In [74]:
#create a dictionary with key as topic_id and values as list of top 10 words. 
def calculate_total_information_gain(model, n_topics, k_words = 10):
  """This function calculates the total information gain of the model.
  args:
    model (Top2Vec): Top2Vec model used in encoding word and document vectors
    n_topics(int): Number of topics to reduce the model to
    k_words(int): To consider top k words (by distance) for each topic while calculating IG
  returns:
    total_IG (float): sum of information gain of all the clusters""" 
  
  #reduce the number of topics in model using hierarchical clustering
  print("Reducing number of topics to", n_topics, "...")
  model.hierarchical_topic_reduction(n_topics)

  #get topic words and ids
  topic_words, similarity_scores, topic_ids = model.get_topics(reduced=True)
  
  #Get the keywords of each topic
  #create a dictionary with key as topic_id and values as list of document ids
  print("Building topic index for keywords...")
  topic_words_dict = {}
  for i in range(len(topic_words)):
    topic_words_dict[i] = topic_words[i][:k_words] #top k words

  #Get the topic number of each document
  #create a dictionary with key as topic_id and values as list of document ids
  print("Building topic index for documents...")
  topic_docs_dict = {}
  for i in range(len(topic_ids)): 
    topic_docs_dict[i] = []

  for i in range(len(documents)):
    topic_id, _, _, _ = model.get_documents_topics([i], reduced=True)
    topic_docs_dict[topic_id[0]].append(i)

  
  #get documents and keywords of each cluster and calculate the information gain
  print("Calculating information gain...")
  total_ig = 0
  for key in tqdm(topic_words_dict):
    words = topic_words_dict[key]
    doc_ids = topic_docs_dict[key]
    docs = np.take(documents, doc_ids)
    topic_ig = calc_IG(words, docs)
    print("\nInformation gain for topic", key, ": ", topic_ig)
    total_ig += topic_ig

  return total_ig

In [ ]:
#plot total information gain
x = []
y_use = []
y_sbert = []
y_doc2vec = []
for i in range(1,302,100):
  if i == 0:
    n_topics = 1
  else:
    n_topics = i
  
  x.append(n_topics)
  ig_use = calculate_total_information_gain(use, n_topics, k_words=10)
  ig_sbert = calculate_total_information_gain(sbert, n_topics, k_words=10)
  ig_doc2vec = calculate_total_information_gain(doc2vec, n_topics, k_words=10)
  y_use.append(ig_use)
  y_sbert.append(ig_sbert)
  y_doc2vec.append(ig_doc2vec)

import matplotlib.pyplot as plt
plt.plot(x,y_use, label="use")
plt.plot(x,y_sbert, label="sbert")
plt.plot(x, y_doc2vec, label="doc2vec")
plt.legend()
plt.show()

Reducing number of topics to 1 ...
Building topic index for keywords...
Building topic index for documents...


  0%|          | 0/1 [00:00<?, ?it/s]

Calculating information gain...


100%|██████████| 1/1 [01:19<00:00, 79.95s/it]


Information gain for topic 0 :  7.575572203030304
Reducing number of topics to 1 ...


In [ ]:
#testing word similarity (to be used in zero shot classification)
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["swimming"], num_topics=3, reduced=False) #swimming, cleanliness #keywords_neg=stop,
for topic in topic_nums:
    model.generate_topic_wordcloud(topic)

In [ ]:
"""Some thoughts:
Inside a cluster: documents should be similar to each other. Perhaps coherence measures that. 

Now, for Mutual Information: It tells us, if given some words, does it tell us anything about the documents?
We kind of changed it slightly because now we are taking top 20 words. We are not even taking the frequency
of these words in fact. Just the set of these. 
So what is the question we are asking within a topic? What kind of behavior we want to model?
Given a list of top n words, we want these words to describe the topic. 
In terms of MI, it will be 0 if 
1)words are not related to the topic, 
2)words are too common that they occur virtually in all the docs, e.g., stop words. 
But what about the words that are representative of the topics and occur in most of the docs in a topic?
They will count less towards MI. e.g. Belgium and Beer in the above example. I guess this is one caveat of this. 
But overall, I think IG should be higher if topic words are represntative.
Also, perhaps there is a way to calculate sentence similarity within topics. A metric of topic coherence based on deep learning.
Comparing words with each after doing TF-IDF, removing stop words, etc.  

Also, some text engineering is required to improve the results. 
1. Named entity recognition. 
2. Sentiment word recognition. 
3. Removing the clusters, whose representative words are most of the above two. (e.g. > 50%). Then,
  merge those documents with the nearest clusters. 

Some thoughts about the coherence score:
The traditional coherence score, while good for obtaining the overall performance of the model doesn't 
take into account the semantic similarity of different words and docs. To truly observe the power of
deep learning models we need semantic similarity score which can compare two documents based on their 
representational similarity, and also take context into account. 
Within a topic, pairwise comparisons can take place. And the sum of those similarity scores is the coherence of that topic. 
The sum of coherence of all the topics is the total coherence of the model. 
1. The simplest way to do it is just to take a vector distance for pairwise comparison. Simply substract the vectors from each other
and take their absolute value. 
2. The more sophisticated way would be to remove stop words from two sentences and and somehow do word by word comparison. But 
sentence-BERT is trained on the semantic similarity task, so it is exactly that task. It should be good to go to measure pairwise 
similarity. But I should read the sentence-BERT paper. 
3. The above two metrics may not work with LDA. So, maybe we will have to retort to the traditional coherence score as well.  

Comments about cluster reduction:
Is hierarchical clustering best way to reduce the number of clusters? What are the alternatives?
""" 


Calculate coherence score

In [65]:
def cos_similarity(vector1, vector2):
  """This functions finds the similarity between doc1 and doc2 using cosine similarity. 
  args:
    vector1(list-like): document embedding 1
    vector2(list-like): document embedding 2
  returns:
    cos_sim(float): cosine similarity score between vector1 and vector2"""
  cos_sim = np.dot(vector1, vector2)/(np.linalg.norm(vector1)*np.linalg.norm(vector2))
  return cos_sim


In [67]:
def coherence(doc_vectors):
  """This function calculates the topic coherence by taking pairwise cosine similarity
  args:
    doc_vectors(list-like): list of document vectors(512 or 768 dim)
  returns:
    coherence_score(float)"""
  num_docs = len(doc_vectors)
  coherence_score = 0
  similarity_matrix = np.zeros((num_docs, num_docs))
  for i in range(num_docs):
    for j in range(num_docs):
      if i!=j: #the similarity of doc to itself is 1 but we don't count it, so in this matrix it is 0
        coherence_score += cos_similarity(doc_vectors[i], doc_vectors[j])

  return coherence_score

In [ ]:
def calculate_total_coherence(model, docs):
  """calculates total coherence for a model. Setences should encoded by the same model to calculate similarity or by some
  external model more suitable for pairwise comparison to make sense. If I encode with sbert, then it will be biased towards sbert""" 

Visualization using tensorboard projector

In [ ]:
#get document vectors and plot using umap and tensorboard
#dim reduction using umap
import umap
umap_embeddings = umap.UMAP(n_neighbors=15,
                            n_components=3, 
                            metric='cosine').fit_transform(model.document_vectors)

In [ ]:
#visualize high dim data in embedding projector
# Set up a logs directory, so Tensorboard knows where to look for files
import numpy as np
import tensorflow as tf
log_dir='./logs/'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# Save the weights we want to analyse as a variable. 
umap_weights = tf.Variable(random.choices(umap_embeddings, k=10000))
# Create a checkpoint from embedding, the filename and key are
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=umap_weights)
checkpoint.save(os.path.join(log_dir, "umap_embedding.ckpt"))

# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
# embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(log_dir, config)

In [ ]:
%tensorboard --logdir ./logs/

In [ ]:
#visualize word embeddings: get rid of stopwords, sentiment words, and take more sentences

In [ ]:
#function to process sentence to remove stopwords and sentiment words
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def process_sentence(s):
  s = s.lower()

  stop = stopwords.words('english')
  sentiment_tags = ['ADJ', 'ADV', 'RB', 'VBZ']


  words = word_tokenize(s)
  #remove stop words
  words = [w for w in words if w not in stop]
  #remove sentiment words
  pos_tags = nltk.pos_tag(words)
  words = [t[0] for t in pos_tags if t[1] not in sentiment_tags]
  s = " ".join(words)
  return s